In [138]:
import dateutil.parser
import os
os.chdir("/home/jovyan/work")

import pandas as pd

from src.utils import send_request

## Create Season Files from MSF

### General Setup

In [139]:
columns=[
    "Date", 
    "Start (ET)", 
    "Visitor/Neutral", 
    "PTS",
    "Home/Neutral", 
    "PTS.1", 
    "OT", 
    "Attend.", 
    "game_type", 
    "game_name",
]

### Season 2017/18

### Regular Season

In [140]:
result1718_reg = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/2017-2018-regular/games.json")

game_reg = result1718_reg.get("games")[0]

game_reg

{'schedule': {'id': 42070,
  'startTime': '2017-10-18T00:00:00.000Z',
  'endedTime': None,
  'awayTeam': {'id': 82, 'abbreviation': 'BOS'},
  'homeTeam': {'id': 86, 'abbreviation': 'CLE'},
  'venue': {'id': 80, 'name': 'Quicken Loans Arena'},
  'venueAllegiance': 'HOME',
  'scheduleStatus': 'NORMAL',
  'originalStartTime': None,
  'delayedOrPostponedReason': None,
  'playedStatus': 'COMPLETED',
  'attendance': None,
  'officials': [],
  'broadcasters': [],
  'weather': None},
 'score': {'currentQuarter': None,
  'currentQuarterSecondsRemaining': None,
  'currentIntermission': None,
  'awayScoreTotal': 99,
  'homeScoreTotal': 102,
  'quarters': [{'quarterNumber': 1, 'awayScore': 19, 'homeScore': 29},
   {'quarterNumber': 2, 'awayScore': 19, 'homeScore': 25},
   {'quarterNumber': 3, 'awayScore': 33, 'homeScore': 18},
   {'quarterNumber': 4, 'awayScore': 28, 'homeScore': 30}]}}

In [141]:
def unmarshall_game(game, game_type="regular_season"):
    row = {}
    date = dateutil.parser.parse(game["schedule"]["startTime"])
    # Date
    row["Date"] = date.strftime("%Y-%m-%d")
    # Start (ET)
    # ! NO TIME
    row["Start (ET)"] = date.strftime("%I:%M %p")
    # Visitor/Neutral
    row["Visitor/Neutral"] = game["schedule"]["awayTeam"]["abbreviation"]
    # PTS
    row["PTS"] = game["score"]["awayScoreTotal"]
    # Home/Neutral
    row["Home/Neutral"] = game["schedule"]["homeTeam"]["abbreviation"]
    # PTS.1
    row["PTS.1"] = game["score"]["homeScoreTotal"]
    # OT
    row["OT"] = 1 if len(game["score"]["quarters"]) > 4 else 0
    # Attend.
    # ! NO INFO
    row["Attend."] = None
    # game_type
    row["game_type"] = game_type
    # game_name
    row["game_name"] = row["Visitor/Neutral"] + " at " + row["Home/Neutral"]
    return row
unmarshall_game(game_reg)

{'Date': '2017-10-18',
 'Start (ET)': '12:00 AM',
 'Visitor/Neutral': 'BOS',
 'PTS': 99,
 'Home/Neutral': 'CLE',
 'PTS.1': 102,
 'OT': 0,
 'Attend.': None,
 'game_type': 'regular_season',
 'game_name': 'BOS at CLE'}

In [142]:
games_reg1718 = list(map(lambda game: unmarshall_game(game), result1718_reg.get("games")))

### Playoff

In [143]:
result1718_po = send_request("https://api.mysportsfeeds.com/v2.0/pull/nba/2018-playoff/games.json")

game_po = result1718_po.get("games")[0]

game_po

{'schedule': {'id': 45890,
  'startTime': '2018-04-14T19:00:00.000Z',
  'endedTime': None,
  'awayTeam': {'id': 106, 'abbreviation': 'SAS'},
  'homeTeam': {'id': 101, 'abbreviation': 'GSW'},
  'venue': {'id': 95, 'name': 'Oracle Arena'},
  'venueAllegiance': 'NEUTRAL',
  'scheduleStatus': 'NORMAL',
  'originalStartTime': None,
  'delayedOrPostponedReason': None,
  'playedStatus': 'COMPLETED',
  'attendance': None,
  'officials': [],
  'broadcasters': [],
  'weather': None},
 'score': {'currentQuarter': None,
  'currentQuarterSecondsRemaining': None,
  'currentIntermission': None,
  'awayScoreTotal': 92,
  'homeScoreTotal': 113,
  'quarters': [{'quarterNumber': 1, 'awayScore': 17, 'homeScore': 28},
   {'quarterNumber': 2, 'awayScore': 24, 'homeScore': 29},
   {'quarterNumber': 3, 'awayScore': 22, 'homeScore': 29},
   {'quarterNumber': 4, 'awayScore': 29, 'homeScore': 27}]}}

In [144]:
unmarshall_game(game_po, game_type="playoff")

{'Date': '2018-04-14',
 'Start (ET)': '07:00 PM',
 'Visitor/Neutral': 'SAS',
 'PTS': 92,
 'Home/Neutral': 'GSW',
 'PTS.1': 113,
 'OT': 0,
 'Attend.': None,
 'game_type': 'playoff',
 'game_name': 'SAS at GSW'}

In [145]:
games_po1718 = list(map(lambda game: unmarshall_game(game), result1718_po.get("games")))

### Merge Season 2017/2018

In [151]:
df_reg = pd.DataFrame.from_records(games_reg1718)
df_po = pd.DataFrame.from_records(games_po1718)

In [152]:
df = df_reg.append(df_po)

In [155]:
df = df[columns]
df.reset_index(drop=True, inplace=True)

In [159]:
df.to_csv("./data/msf/season_files/season1718.csv", index=False)